In [1]:

from func_Info import *
import pandas as pd
import numpy as np
from itertools import combinations

In [2]:
path="../data_extract/"
ratid=22098
chunksize=20 #in minutes
rats=[20382,24101,21012,22295,20630,22098,23783,24116]

In [3]:
dt=pd.read_json(path+"Rat_"+str(ratid)+"_RECdata_extracted.json");

In [4]:
dt.columns

Index(['RAT_ID', 'REC_GID', 'REC_duration', 'REC_timerange', 'REC_Date',
       'N_DIDs', 'Neu_IDs', 'N_GIDs', 'N_LOCs', 'task', 'U_GIDs',
       'U_spiketimes'],
      dtype='object')

70

# Checking all combinations

In [18]:
nRec=2;
allD=[];

#checking if the duration is better than chunk size
if dt.iloc[nRec].REC_duration > (chunksize*60): #in seconds
    #segrecating CA1 and SUB units in the recording
    loci = {'CA1': [], 'SUB': [], 'others': []}
    for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
        if 'CA1' in v:
            loci['CA1'].append(s)
        elif 'SUB' in v:
            loci['SUB'].append(s)
        else:
            loci['others'].append(s)
    print(loci)
    no_CA1=len(loci['CA1'])
    no_SUB=len(loci['SUB'])        
    #SUB neurons loop
    if no_SUB!= 0:
        if no_SUB == 1:
            allD=adding_rows(allD,'SUB',False);
        else:
            allD=adding_rows(allD,'SUB',True);
    #CA1 Loop
    if no_CA1 != 0:
        if no_CA1 == 1:
            allD=adding_rows(allD,'CA1',False);
        else:
            allD=adding_rows(allD,'CA1',True);
pd.DataFrame(allD)    

{'CA1': [], 'SUB': [8488, 8490, 8682, 8688, 8692, 8695, 8697, 8699, 8701, 8703], 'others': []}


,RAT_ID,Rec_GID,Rtask,U_LOC,U1_GID,OP,U2_GID,Nspikes,MSR,MHK,dt_MHK,OHK,OHS,dt_OHK,max_HSHK
0,22098,6456,fm,SUB,8488,NA,8488,9778,0.272050,0.446676,9.135425,0.213495,0.878982,0.253393,1.092477
1,22098,6456,fm,SUB,8490,NA,8490,6756,0.275586,0.453334,9.135425,0.220777,0.880605,0.302249,1.101382
2,22098,6456,fm,SUB,8682,NA,8682,982,0.294894,0.493372,9.135425,0.414646,0.744537,0.979118,1.159183
3,22098,6456,fm,SUB,8688,NA,8688,884,0.295243,0.490169,14.618976,0.368099,0.789824,0.923233,1.157924
4,22098,6456,fm,SUB,8692,NA,8692,982,0.294894,0.493372,9.135425,0.414646,0.744537,0.979118,1.159183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,22098,6456,fm,SUB,8699,OR,8703,1792,0.297948,0.482277,6.420778,0.395621,0.774986,0.576930,1.170607
141,22098,6456,fm,SUB,8699,XOR,8703,1704,0.297983,0.485647,6.809436,0.392883,0.774867,0.611852,1.167749
142,22098,6456,fm,SUB,8701,AND,8703,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,22098,6456,fm,SUB,8701,OR,8703,1525,0.296215,0.493824,13.784577,0.387088,0.775896,0.729822,1.162984


In [5]:
def empty_formatted(u1,u2,op,nspike,LOC):

    rows={'RAT_ID': dt.iloc[nRec].RAT_ID,
        'Rec_GID' : dt.iloc[nRec].REC_GID, 
        'Rtask'   : dt.iloc[nRec].task,
        
        'U_LOC'   :LOC,
        'U1_GID':  u1,
        'OP': op,
        'U2_GID':u2,
 
        'Nspikes':nspike,
        'MSR':np.nan,
        'MHK':np.nan,
        'dt_MHK':np.nan,
        'OHK':np.nan,
        'OHS':np.nan,
        'dt_OHK':np.nan,
        'max_HSHK':np.nan
        };
    return rows;

def single_formatted(spike_train,st,u,LOC):
    nspikes,msr,OHKs,OHSs,maxHkpHs,Tohk,MHKs,Tmhks=MSR(spike_train,st,st+(chunksize*60),200)
    
    rows={'RAT_ID': dt.iloc[nRec].RAT_ID,
        'Rec_GID' : dt.iloc[nRec].REC_GID, 
        'Rtask'   : dt.iloc[nRec].task,
        
        'U_LOC'   :LOC,
        'U1_GID':  u,
        'OP': 'NA',
        'U2_GID':u,
 
        'Nspikes':nspikes,
        'MSR':msr,
        'MHK':MHKs,
        'dt_MHK':Tmhks,
        'OHK':OHKs,
        'OHS':OHSs,
        'dt_OHK':Tohk,
        'max_HSHK':maxHkpHs
        };
    return rows;
def muli_formatted(spike_train,st,u1,u2,op,LOC):
    #for The spike train of length = chunksize
    nspikes,msr,OHKs,OHSs,maxHkpHs,Tohk,MHKs,Tmhks=MSR(spike_train,st,st+(chunksize*60),200)
    
    rows={'RAT_ID': dt.iloc[nRec].RAT_ID,
        'Rec_GID' : dt.iloc[nRec].REC_GID, 
        'Rtask'   : dt.iloc[nRec].task,
        
        'U_LOC'   :LOC,
        'U1_GID':  u1,
        'OP': op,
        'U2_GID':u2,
 
        'Nspikes':nspikes,
        'MSR':msr,
        'MHK':MHKs,
        'dt_MHK':Tmhks,
        'OHK':OHKs,
        'OHS':OHSs,
        'dt_OHK':Tohk,
        'max_HSHK':maxHkpHs
        };
    return rows;

In [6]:

def adding_rows(datas,location,mul_unit):
   bin_time=0.001 #1 milisec in seconds time resolution of binning for logical operations
   st,en=dt.iloc[nRec].REC_timerange
   edges = np.arange(st, en + bin_time, bin_time)
   # Binarization of spike train lambda function
   bined_spikes = lambda unit_id: np.array([False if x == 0 else True for x in np.histogram(dt.iloc[nRec].U_spiketimes[dt.iloc[nRec].U_GIDs.index(unit_id)], bins=edges)[0]])

   #For Single Units
   for items in loci[location]:
      sptr=np.array(dt.iloc[nRec].U_spiketimes[dt.iloc[nRec].U_GIDs.index(items)]); #spiketrain
      if len(np.where(sptr <= st+(chunksize*60))[0]) > 1:
         datas.append(single_formatted(sptr,st,items,location))
      else:
         datas.append(empty_formatted(items,items,'NA',len(np.where(sptr <= st+(chunksize*60))[0]),location))
   if mul_unit:   
      #For combined Units
      for items in list(combinations(loci[location], 2)):
         uid1=items[0];uid2=items[1]; #unit id
         #print(uid1," ",uid2)
         
         #binarizing the spike train with 1 ms time resolution for logical operation
         bst1=bined_spikes(uid1);bst2=bined_spikes(uid2);
         # & is the element wise (bit wise) AND ; where retuns indices of the True after bool operation;
         #finally select from the time array from st to en in 1 ms resolution to create spike timestamps
            
         # AND operation
         and_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 & bst2)]
         if len(np.where(and_spike_train <= st+(chunksize*60))[0]) > 1:
            datas.append(muli_formatted(and_spike_train,st,uid1,uid2,'AND',location))
         else:
            datas.append(empty_formatted(uid1,uid2,'AND',len(np.where(and_spike_train <= st+(chunksize*60))[0]),location))
         
         # OR operation
         or_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 | bst2)]
         if len(np.where(or_spike_train <= st+(chunksize*60))[0]) > 1:
            datas.append(muli_formatted(or_spike_train,st,uid1,uid2,'OR',location))
         else:
            datas.append(empty_formatted(uid1,uid2,'OR',len(np.where(or_spike_train <= st+(chunksize*60))[0]),location))
         
         # XOR operation
         xor_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 ^ bst2)]
         if len(np.where(xor_spike_train <= st+(chunksize*60))[0]) > 1:
            datas.append(muli_formatted(xor_spike_train,st,uid1,uid2,'XOR',location))
         else:
            datas.append(empty_formatted(uid1,uid2,'XOR',len(np.where(xor_spike_train <= st+(chunksize*60))[0]),location))  
   return datas 


  

# Checking individual recordings 

In [33]:
nRec=2;

#segrecating CA1 and SUB units in the recording
loci = {'CA1': [], 'SUB': [], 'others': []}
for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
    if 'CA1' in v:
        loci['CA1'].append(s)
    elif 'SUB' in v:
        loci['SUB'].append(s)
    else:
        loci['others'].append(s)
        
#checking if the duration is better than chunk size
dt.iloc[nRec].REC_duration > (chunksize*60) #in seconds


True

In [34]:
dt.iloc[nRec]

RAT_ID                                                       22098
REC_GID                                                       6456
REC_duration                                            1600.93825
REC_timerange                     [4104.714144, 5705.652394000001]
REC_Date                                                01/05/2016
N_DIDs           [286, 287, 327, 329, 330, 332, 333, 334, 335, ...
Neu_IDs          [423, 424, 467, 469, 470, 472, 475, 476, 477, ...
N_GIDs           [13293, 13294, 13334, 13336, 13337, 13339, 133...
N_LOCs           [[PROX, SUB], [PROX, SUB], [PROX, SUB], [PROX,...
task                                                            fm
U_GIDs           [8488, 8490, 8682, 8688, 8692, 8695, 8697, 869...
U_spiketimes     [[4104.7611, 4104.8735, 4104.9446, 4105.1203, ...
Name: 2, dtype: object

In [73]:
uid1=8682; #unit id
uid2=8699; #unit id

bin_time=0.001 #1 milisec in seconds time resolution of binning for logical operations

In [74]:
st,en=dt.iloc[nRec].REC_timerange
edges = np.arange(st, en + bin_time, bin_time)
#binarizing the spike train with 1 ms time resolution for logical operation
bined_spikes = lambda unit_id: np.array([False if x == 0 else True for x in np.histogram(dt.iloc[nRec].U_spiketimes[dt.iloc[nRec].U_GIDs.index(unit_id)], bins=edges)[0]])

bst1=bined_spikes(uid1)
bst2=bined_spikes(uid2)


In [75]:
# AND operation
and_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 & bst2)]


# OR operation
or_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 | bst2)]


# XOR operation
xor_spike_train = np.arange(st,en+bin_time,bin_time)[np.where(bst1 ^ bst2)]


# & is the element wise (bit wise) AND ; where retuns indices of the True after bool operation;
#finally select from the time array from st to en in 1 ms resolution to create spike timestamps

## Checking the MSR-Python

In [77]:
print(MSR(and_spike_train,st,en,200))
#for The spike train of length = chunksize
print(MSR(and_spike_train,st,st+(chunksize*60),200))

(2, 0.0, 0.0, 1.0, 1.0, 1e-11, 0.0, 1e-11)
(0, 0.0, 0.0, 1.0, 1.0, 1e-11, 0.0, 1e-11)


/home/sapta/Dropbox/Saptarshi_Postdoc_Projects/Project_MSR/Codes_4/data_bool/../lib/func_MSR.py:25: RuntimeWarning: divide by zero encountered in log
  ((freq * vals) / M) * (np.log(vals/M) / np.log(M)) # Entropy calculation on Base M
/home/sapta/Dropbox/Saptarshi_Postdoc_Projects/Project_MSR/Codes_4/data_bool/../lib/func_MSR.py:29: RuntimeWarning: divide by zero encountered in log
  ((freq * vals) / M) * (np.log( (freq * vals) /M) / np.log(M)) # Entropy calculation on Base M


## Relating with the neuron dataframe

In [ ]:
# df=pd.read_json(path+"Rat_"+str(ratid)+"_data_extracted.json");

# unit_GID_indices = {ele : (row_index, ele_index)
#                     for row_index, row in df.iterrows()
#                     for ele_index, ele in enumerate(row['U_GID'])}


In [ ]:
# nRec=2;

# #segrecating CA1 and SUB units in the recording
# loci = {'CA1': [], 'SUB': [], 'others': []}
# for (s, v) in zip(dt.iloc[nRec].U_GIDs, dt.iloc[nRec].N_LOCs):
#     if 'CA1' in v:
#         loci['CA1'].append(s)
#     elif 'SUB' in v:
#         loci['SUB'].append(s)
#     else:
#         loci['others'].append(s)
        
# #checking if the duration is better than chunk size
# dt.iloc[nRec].REC_duration > (chunksize*60) #in seconds

In [ ]:
# dt.iloc[nRec]

In [ ]:
# uid=8488; #unit id
# bin_time=0.001 #1 milisec in seconds time resolution of binning for logical operations
# nu,uni=unit_GID_indices[uid];

In [ ]:

# # asserting matching REC date
# test1= df.loc[nu].REC_Date[uni] == dt.iloc[nRec].REC_Date;
# # finding the index of the unit in the recording
# find_index = lambda lst, target: next((index for index, value in enumerate(lst) if value == target), None)
# #asserting matching N_GID
# test2= df.loc[nu].N_GID ==dt.iloc[nRec].N_GIDs[find_index(dt.iloc[nRec].U_GIDs,uid)];
# #asserting matching N_DID
# test3=df.loc[nu].N_DID == dt.iloc[nRec].N_DIDs[find_index(dt.iloc[nRec].U_GIDs,uid)];
# # extrating the ID of the neuron
# Neu_ID= dt.iloc[nRec].Neu_IDs[find_index(dt.iloc[nRec].U_GIDs,uid)]

# print(test1,test2,test3,Neu_ID)